# ML HW4
### 106598018 萬俊瑋

**1. In the SOFM network, the learning rate is a function of time and distance between the current node and the BMU. If η0 = σ0 = 0.1, λ = 10, find the number of required iteration such that the learning rate of nodes next to BMU is less than 0.001. A node is next to BMU is located at (x0 ± 1, y0) or (x0, y0 ±1) where (x0, y0) as the coordinate of the BMU.**

In [1]:
import numpy as np

learningRate = 1
t = 1
while learningRate >= 0.001:
    learningRate = 0.1*np.exp(t/10)*np.exp(-1/(2*(0.1*np.exp(-t/10))**2))
    t += 1
print('Iteration:' + str(t - 1), learningRate)

Iteration:1 3.318986956663983e-28


**2. We mentioned that the parameter α in GMM was computed based on the Lagrange multipliers. Show that αj = (1/n)∑βj(xi) as given in the PPT notes.**

[http://www.cs.nccu.edu.tw/~whliao/acv2008/08gmm.pdf](http://www.cs.nccu.edu.tw/~whliao/acv2008/08gmm.pdf)

**3. We have analytically solved the following problem: Maximize f(x, y) = x + y subject to x2 + y2 = 1. Write a gradient descent program to find the solution numerically. Note that to find the maximum point, you need to follow the gradient (instead of negative gradient). Compare your numerical results with analytical results.**

In [2]:
def numericalGradient(F, X):
    h = 1e-6
    G = np.zeros_like(X)
    for i in range(len(X)):
        oriX = X[i]
        X[i] = oriX + h
        FXh1 = F(X)
        X[i] = oriX - h
        FXh2 = F(X)
        G[i] = (FXh1 - FXh2)/(2*h)
        X[i] = oriX
    return G

def gradientDescent(F, X, eta=0.1, maxEpoch=1000):
    epoch = 1
    while True:
        G = numericalGradient(F, X)
        X += eta*G
        if np.sqrt(np.sum(G**2)) < 1e-6 or epoch >= maxEpoch:
            break
        epoch += 1
    return X, epoch


if __name__=='__main__':
    def myLagrangeMutipliers(X):
        x = X[0]
        y = X[1]
        # magic number of my analytical results
        l = -1*np.sqrt(2)/2
        return x + y + l*(x**2 + y**2 - 1)
    
    X = np.random.uniform(0., 1., 3)
    X, epoch = gradientDescent(myLagrangeMutipliers, X)
    
    print('epoch:', epoch, 'The max number of f(x, y) = x + y subject to x^2 + y^2 = 1:', X[0] + X[1])

epoch: 83 The max number of f(x, y) = x + y subject to x^2 + y^2 = 1: 1.4142141999225553


**4. Compute the complete update (back propagation) equations for all weights (w1 ~ w8) in the following neural networks. The activation function is sigmoid, the loss function is MSE, and the desired outputs are d1 and d2.**
![](https://github.com/wei840222/ML-HW/blob/develop/images/NN.JPG?raw=true)

**5. Write a program to implement the neural network with your back propagation equations in problem 4. To test your network, train it to distinguish the classes of versicolor and virginica in the Iris dataset using only the third and fourth features
(i.e., petal length and petal width) as the inputs. As usual, use 70% of the data for training and the rest for testing. Repeat the experiments 10 times to find the average accuracy. During training, set the desired output as 0.9 for in class data
and 0.1 for out of class data. Don’t forget to use random numbers as the initial weights.**

In [3]:
# Dependencies
import tensorflow as tf
import pandas as pd
import numpy as np

# Make results reproducible
seed = 1234
np.random.seed(seed)
tf.set_random_seed(seed)

# Loading the dataset
dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
dataset = pd.get_dummies(dataset, columns=[4]) # One Hot Encoding
values = list(dataset.columns.values)

y = dataset[values[-3:]]
y = np.array(y, dtype='float32')
X = dataset[values[:4]]
X = np.array(X, dtype='float32')

# Shuffle Data
indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]
y_values = y[indices]

# Creating a Train and a Test Dataset
test_size = 10
X_test = X_values[-test_size:]
X_train = X_values[:-test_size]
y_test = y_values[-test_size:]
y_train = y_values[:-test_size]

# Session
sess = tf.Session()

# Interval / Epochs
interval = 50
epoch = 500

# Initialize placeholders
X_data = tf.placeholder(shape=[None, 4], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 3], dtype=tf.float32)

# Input neurons : 4
# Hidden neurons : 8
# Output neurons : 3
hidden_layer_nodes = 8

# Create variables for Neural Network layers
w1 = tf.Variable(tf.random_normal(shape=[4,hidden_layer_nodes])) # Inputs -> Hidden Layer
b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))   # First Bias
w2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes,3])) # Hidden layer -> Outputs
b2 = tf.Variable(tf.random_normal(shape=[3]))   # Second Bias

# Operations
hidden_output = tf.nn.relu(tf.add(tf.matmul(X_data, w1), b1))
final_output = tf.nn.softmax(tf.add(tf.matmul(hidden_output, w2), b2))

# Cost Function
loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=0))

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)

# Training
print('Training the model...')
for i in range(1, (epoch + 1)):
    sess.run(optimizer, feed_dict={X_data: X_train, y_target: y_train})
    if i % interval == 0:
        print('Epoch', i, '|', 'Loss:', sess.run(loss, feed_dict={X_data: X_train, y_target: y_train}))

# Prediction
print()
for i in range(len(X_test)):
    print('Actual:', y_test[i], 'Predicted:', np.rint(sess.run(final_output, feed_dict={X_data: [X_test[i]]})))

/home/wei/Anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Training the model...
Epoch 50 | Loss: 26.049301
Epoch 100 | Loss: 13.553838
Epoch 150 | Loss: 9.704756
Epoch 200 | Loss: 7.478199
Epoch 250 | Loss: 6.327774
Epoch 300 | Loss: 5.5770936
Epoch 350 | Loss: 5.0523667
Epoch 400 | Loss: 4.6667256
Epoch 450 | Loss: 4.358085
Epoch 500 | Loss: 4.0857043

Actual: [0. 0. 1.] Predicted: [[0. 0. 1.]]
Actual: [1. 0. 0.] Predicted: [[1. 0. 0.]]
Actual: [0. 0. 1.] Predicted: [[0. 0. 1.]]
Actual: [1. 0. 0.] Predicted: [[1. 0. 0.]]
Actual: [1. 0. 0.] Predicted: [[1. 0. 0.]]
Actual: [0. 0. 1.] Predicted: [[0. 0. 1.]]
Actual: [0. 0. 1.] Predicted: [[0. 0. 1.]]
Actual: [0. 1. 0.] Predicted: [[0. 1. 0.]]
Actual: [1. 0. 0.] Predicted: [[1. 0. 0.]]
Actual: [1. 0. 0.] Predicted: [[1. 0. 0.]]
